<a href="https://colab.research.google.com/github/canabica/Gemma-2-Alpaca-9B/blob/main/Gemma_2_Alpaca_9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Gemma2-9b is trained on 8 trillion tokens! Gemma2-27b is 13 trillion!**

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",          # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [5]:
import json

# JSON corrigido com aspas escapadas
json_string = """
[
{
"instruction": "Analisar",
"input": "o efeito de um blend de Cannabis Sativa e predominantemente Indica na aprendizagem de idiomas",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Sativa-Indica\\" -> \\"Aprendizagem de Idiomas\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Sativa-Indica\\" -> \\"Foco\\" [label=\\"Aumenta (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Sativa-Indica\\" -> \\"Ansiedade\\" [label=\\"Reduz (NCA)\\", color=\\"blue\\"];\\n \\"Foco\\" -> \\"Aprendizagem de Idiomas\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n \\"Ansiedade\\" -> \\"Aprendizagem de Idiomas\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o consumo de um blend predominantemente Sativa e a capacidade de memorização de longo prazo",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Memorização de Longo Prazo\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Concentração\\" [label=\\"Aumenta (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Motivação\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Concentração\\" -> \\"Memorização de Longo Prazo\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n \\"Motivação\\" -> \\"Memorização de Longo Prazo\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "a influência do teor de THC e CBD em um blend Indica na retenção de informações em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Teor de THC (Blend Indica)\\" -> \\"Retenção de Informações\\" [label=\\"Modula (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Teor de CBD (Blend Indica)\\" -> \\"Retenção de Informações\\" [label=\\"Modula (PLS-SEM)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o uso de um blend predominantemente Indica e a ansiedade em estudantes universitários",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Indica\\" -> \\"Ansiedade\\" [label=\\"Reduz (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Indica\\" -> \\"Relaxamento\\" [label=\\"Aumenta (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Relaxamento\\" -> \\"Ansiedade\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o perfil de terpenos em um blend Sativa e o desempenho em provas de múltipla escolha",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Perfil de Terpenos (Blend Sativa)\\" -> \\"Desempenho em Provas\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "o impacto do método de consumo (inalação vs. ingestão) na concentração de canabinoides em um blend Indica e seu efeito na aprendizagem",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Método de Consumo\\" -> \\"Concentração de Canabinoides (Blend Indica)\\" [label=\\"Influencia (PLS-SEM)\\", color=\\"green\\"];\\n \\"Concentração de Canabinoides (Blend Indica)\\" -> \\"Aprendizagem\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o consumo de um blend predominantemente Sativa e a capacidade de foco em estudantes com TDAH",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Foco\\" [label=\\"Aumenta (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Impulsividade\\" [label=\\"Reduz (NCA)\\", color=\\"blue\\"];\\n \\"Impulsividade\\" -> \\"Foco\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o uso de um blend Indica e a retenção de informações em palestras",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Indica\\" -> \\"Retenção de Informações\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Indica\\" -> \\"Relaxamento\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Relaxamento\\" -> \\"Retenção de Informações\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "a influência da dosagem em um blend predominantemente Indica e seu efeito na capacidade de aprendizagem",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Dosagem (Blend Predominantemente Indica)\\" -> \\"Aprendizagem\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o uso de um blend Sativa e a performance em trabalhos em grupo",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Sativa\\" -> \\"Performance em Grupo\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Sativa\\" -> \\"Comunicação\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Comunicação\\" -> \\"Performance em Grupo\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "o efeito de um blend de Cannabis Sativa e Indica na capacidade de resolução de problemas matemáticos",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Sativa-Indica\\" -> \\"Resolução de Problemas\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Sativa-Indica\\" -> \\"Pensamento Lógico\\" [label=\\"Estimula (NCA)\\", color=\\"blue\\"];\\n \\"Pensamento Lógico\\" -> \\"Resolução de Problemas\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o consumo de um blend predominantemente Sativa e a fluência verbal em debates",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Fluência Verbal\\" [label=\\"Aumenta (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Articulação de Ideias\\" [label=\\"Melhora (NCA)\\", color=\\"blue\\"];\\n \\"Articulação de Ideias\\" -> \\"Fluência Verbal\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "a influência do teor de CBD em um blend Indica na ansiedade durante apresentações em público",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Teor de CBD (Blend Indica)\\" -> \\"Ansiedade\\" [label=\\"Reduz (PLS-SEM)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o uso de um blend predominantemente Indica e a qualidade do sono em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Indica\\" -> \\"Qualidade do Sono\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Indica\\" -> \\"Relaxamento\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Relaxamento\\" -> \\"Qualidade do Sono\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o perfil de terpenos em um blend Sativa e a capacidade de concentração em leituras",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Perfil de Terpenos (Blend Sativa)\\" -> \\"Concentração\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "o impacto do método de extração na concentração de canabinoides em um blend Indica e seu efeito na memória",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Método de Extração\\" -> \\"Concentração de Canabinoides (Blend Indica)\\" [label=\\"Influencia (PLS-SEM)\\", color=\\"green\\"];\\n \\"Concentração de Canabinoides (Blend Indica)\\" -> \\"Memória\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o uso de um blend predominantemente Sativa e a criatividade em redações",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Criatividade em Redações\\" [label=\\"Aumenta (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Pensamento Divergente\\" [label=\\"Estimula (NCA)\\", color=\\"blue\\"];\\n \\"Pensamento Divergente\\" -> \\"Criatividade em Redações\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o uso de um blend Indica e a redução do estresse em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Indica\\" -> \\"Estresse\\" [label=\\"Reduz (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Indica\\" -> \\"Relaxamento\\" [label=\\"Promove (NCA)\\", color=\\"blue\\"];\\n \\"Relaxamento\\" -> \\"Estresse\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "a influência da genética da planta na composição química de um blend predominantemente Sativa e seu efeito na capacidade de foco",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Genética da Planta\\" -> \\"Composição Química (Blend Predominantemente Sativa)\\" [label=\\"Influencia (PLS-SEM)\\", color=\\"green\\"];\\n \\"Composição Química (Blend Predominantemente Sativa)\\" -> \\"Foco\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o uso de um blend Sativa e a performance em atividades que exigem coordenação motora fina, como desenho",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Sativa\\" -> \\"Coordenação Motora Fina\\" [label=\\"Modula (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Sativa\\" -> \\"Concentração\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Concentração\\" -> \\"Coordenação Motora Fina\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "o efeito de um blend de Cannabis Sativa e Indica na memória de curto prazo em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Sativa-Indica\\" -> \\"Memória de Curto Prazo\\" [label=\\"Modula (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Sativa-Indica\\" -> \\"Foco\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Foco\\" -> \\"Memória de Curto Prazo\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o consumo de um blend predominantemente Sativa e o desempenho em tarefas de atenção seletiva em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Atenção Seletiva\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Impulsividade\\" [label=\\"Reduz (NCA)\\", color=\\"blue\\"];\\n \\"Impulsividade\\" -> \\"Atenção Seletiva\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "a influência do teor de THC e CBD em um blend Indica na capacidade de aprendizagem verbal em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Teor de THC (Blend Indica)\\" -> \\"Aprendizagem Verbal\\" [label=\\"Modula (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Teor de CBD (Blend Indica)\\" -> \\"Aprendizagem Verbal\\" [label=\\"Modula (PLS-SEM)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o uso de um blend predominantemente Indica e a performance em tarefas de memória espacial em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Indica\\" -> \\"Memória Espacial\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Indica\\" -> \\"Relaxamento\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Relaxamento\\" -> \\"Memória Espacial\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o perfil de terpenos em um blend Sativa e a capacidade de resolução de problemas complexos em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Perfil de Terpenos (Blend Sativa)\\" -> \\"Resolução de Problemas Complexos\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "o impacto do método de cultivo na concentração de canabinoides em um blend Indica e seu efeito na memória de longo prazo em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Método de Cultivo\\" -> \\"Concentração de Canabinoides (Blend Indica)\\" [label=\\"Influencia (PLS-SEM)\\", color=\\"green\\"];\\n \\"Concentração de Canabinoides (Blend Indica)\\" -> \\"Memória de Longo Prazo\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o consumo de um blend predominantemente Sativa e a capacidade de aprendizado visuoespacial em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Aprendizado Visuoespacial\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Predominantemente Sativa\\" -> \\"Concentração\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Concentração\\" -> \\"Aprendizado Visuoespacial\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Analisar",
"input": "a correlação entre o uso de um blend Indica e a capacidade de retenção de informações em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Indica\\" -> \\"Retenção de Informações\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Indica\\" -> \\"Relaxamento\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Relaxamento\\" -> \\"Retenção de Informações\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
},
{
"instruction": "Avaliar",
"input": "a influência da proporção THC:CBD em um blend predominantemente Indica e seu efeito na capacidade de aprendizagem motora em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Proporção THC:CBD (Blend Predominantemente Indica)\\" -> \\"Aprendizagem Motora\\" [label=\\"Modula (PLS-SEM)\\", color=\\"green\\"];\\n }"
},
{
"instruction": "Modelar",
"input": "a relação entre o uso de um blend Sativa e a performance em tarefas de atenção sustentada em estudantes",
"output": "digraph {\\n node [shape=circle, fontname=\\"Calibri Bold\\"];\\n edge [fontname=\\"Calibri Bold\\"];\\n \\"Blend Sativa\\" -> \\"Atenção Sustentada\\" [label=\\"Melhora (PLS-SEM)\\", color=\\"blue\\"];\\n \\"Blend Sativa\\" -> \\"Energia\\" [label=\\"Aumenta (NCA)\\", color=\\"blue\\"];\\n \\"Energia\\" -> \\"Atenção Sustentada\\" [label=\\"Influencia (NCA)\\", color=\\"blue\\"];\\n }"
}
]
"""

# Carregar o JSON
data = json.loads(json_string)

# Imprimir a instrução do primeiro item
print(data[0]["instruction"])

# Imprimir o input do segundo item
print(data[1]["input"])

# Iterar sobre todos os itens e imprimir as informações
for item in data:
    print(f"Instrução: {item['instruction']}")
    print(f"Input: {item['input']}")
    print(f"Output: {item['output']}")
    print("-" * 20)

Analisar
a correlação entre o consumo de um blend predominantemente Sativa e a capacidade de memorização de longo prazo
Instrução: Analisar
Input: o efeito de um blend de Cannabis Sativa e predominantemente Indica na aprendizagem de idiomas
Output: digraph {
 node [shape=circle, fontname="Calibri Bold"];
 edge [fontname="Calibri Bold"];
 "Blend Sativa-Indica" -> "Aprendizagem de Idiomas" [label="Melhora (PLS-SEM)", color="blue"];
 "Blend Sativa-Indica" -> "Foco" [label="Aumenta (PLS-SEM)", color="blue"];
 "Blend Sativa-Indica" -> "Ansiedade" [label="Reduz (NCA)", color="blue"];
 "Foco" -> "Aprendizagem de Idiomas" [label="Influencia (NCA)", color="blue"];
 "Ansiedade" -> "Aprendizagem de Idiomas" [label="Influencia (NCA)", color="blue"];
 }
--------------------
Instrução: Analisar
Input: a correlação entre o consumo de um blend predominantemente Sativa e a capacidade de memorização de longo prazo
Output: digraph {
 node [shape=circle, fontname="Calibri Bold"];
 edge [fontname="Calibri 

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset
import json

# Base de treinamento inserida diretamente no código
train_data = [
    {
        "instruction": "Analisar",
        "input": "o efeito de um blend de Cannabis Sativa e predominantemente Indica na aprendizagem de idiomas",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Aprendizagem de Idiomas\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Foco\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Ansiedade\\\" [label=\\\"Reduz (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Foco\\\" -> \\\"Aprendizagem de Idiomas\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Ansiedade\\\" -> \\\"Aprendizagem de Idiomas\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o consumo de um blend predominantemente Sativa e a capacidade de memorização de longo prazo",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Memorização de Longo Prazo\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Concentração\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Motivação\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Concentração\\\" -> \\\"Memorização de Longo Prazo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Motivação\\\" -> \\\"Memorização de Longo Prazo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência do teor de THC e CBD em um blend Indica na retenção de informações em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Teor de THC (Blend Indica)\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Teor de CBD (Blend Indica)\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Indica e a ansiedade em estudantes universitários",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Ansiedade\\\" [label=\\\"Reduz (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Ansiedade\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o perfil de terpenos em um blend Sativa e o desempenho em provas de múltipla escolha",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Perfil de Terpenos (Blend Sativa)\\\" -> \\\"Desempenho em Provas\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "o impacto do método de consumo (inalação vs. ingestão) na concentração de canabinoides em um blend Indica e seu efeito na aprendizagem",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Método de Consumo\\\" -> \\\"Concentração de Canabinoides (Blend Indica)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Concentração de Canabinoides (Blend Indica)\\\" -> \\\"Aprendizagem\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o consumo de um blend predominantemente Sativa e a capacidade de foco em estudantes com TDAH",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Foco\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Impulsividade\\\" [label=\\\"Reduz (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Impulsividade\\\" -> \\\"Foco\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o uso de um blend Indica e a retenção de informações em palestras",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Indica\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência da dosagem em um blend predominantemente Indica e seu efeito na capacidade de aprendizagem",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Dosagem (Blend Predominantemente Indica)\\\" -> \\\"Aprendizagem\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend Sativa e a performance em trabalhos em grupo",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Performance em Grupo\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Comunicação\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Comunicação\\\" -> \\\"Performance em Grupo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "o efeito de um blend de Cannabis Sativa e Indica na capacidade de resolução de problemas matemáticos",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Resolução de Problemas\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Pensamento Lógico\\\" [label=\\\"Estimula (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Pensamento Lógico\\\" -> \\\"Resolução de Problemas\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o consumo de um blend predominantemente Sativa e a fluência verbal em debates",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Fluência Verbal\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Articulação de Ideias\\\" [label=\\\"Melhora (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Articulação de Ideias\\\" -> \\\"Fluência Verbal\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência do teor de CBD em um blend Indica na ansiedade durante apresentações em público",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Teor de CBD (Blend Indica)\\\" -> \\\"Ansiedade\\\" [label=\\\"Reduz (PLS-SEM)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Indica e a qualidade do sono em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Qualidade do Sono\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Qualidade do Sono\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o perfil de terpenos em um blend Sativa e a capacidade de concentração em leituras",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Perfil de Terpenos (Blend Sativa)\\\" -> \\\"Concentração\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "o impacto do método de extração na concentração de canabinoides em um blend Indica e seu efeito na memória",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Método de Extração\\\" -> \\\"Concentração de Canabinoides (Blend Indica)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Concentração de Canabinoides (Blend Indica)\\\" -> \\\"Memória\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Sativa e a criatividade em redações",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Criatividade em Redações\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Pensamento Divergente\\\" [label=\\\"Estimula (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Pensamento Divergente\\\" -> \\\"Criatividade em Redações\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o uso de um blend Indica e a redução do estresse em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Indica\\\" -> \\\"Estresse\\\" [label=\\\"Reduz (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Promove (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Estresse\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência da genética da planta na composição química de um blend predominantemente Sativa e seu efeito na capacidade de foco",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Genética da Planta\\\" -> \\\"Composição Química (Blend Predominantemente Sativa)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Composição Química (Blend Predominantemente Sativa)\\\" -> \\\"Foco\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend Sativa e a performance em atividades que exigem coordenação motora fina, como desenho",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Coordenação Motora Fina\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Concentração\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Concentração\\\" -> \\\"Coordenação Motora Fina\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "o efeito de um blend de Cannabis Sativa e Indica na memória de curto prazo em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Memória de Curto Prazo\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Foco\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Foco\\\" -> \\\"Memória de Curto Prazo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o consumo de um blend predominantemente Sativa e o desempenho em tarefas de atenção seletiva em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Atenção Seletiva\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Impulsividade\\\" [label=\\\"Reduz (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Impulsividade\\\" -> \\\"Atenção Seletiva\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência do teor de THC e CBD em um blend Indica na capacidade de aprendizagem verbal em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Teor de THC (Blend Indica)\\\" -> \\\"Aprendizagem Verbal\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Teor de CBD (Blend Indica)\\\" -> \\\"Aprendizagem Verbal\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Indica e a performance em tarefas de memória espacial em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Memória Espacial\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Memória Espacial\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o perfil de terpenos em um blend Sativa e a capacidade de resolução de problemas complexos em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Perfil de Terpenos (Blend Sativa)\\\" -> \\\"Resolução de Problemas Complexos\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "o impacto do método de cultivo na concentração de canabinoides em um blend Indica e seu efeito na memória de longo prazo em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Método de Cultivo\\\" -> \\\"Concentração de Canabinoides (Blend Indica)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Concentração de Canabinoides (Blend Indica)\\\" -> \\\"Memória de Longo Prazo\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o consumo de um blend predominantemente Sativa e a capacidade de aprendizado visuoespacial em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Aprendizado Visuoespacial\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Concentração\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Concentração\\\" -> \\\"Aprendizado Visuoespacial\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o uso de um blend Indica e a capacidade de retenção de informações em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Indica\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência da proporção THC:CBD em um blend predominantemente Indica e seu efeito na capacidade de aprendizagem motora em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Proporção THC:CBD (Blend Predominantemente Indica)\\\" -> \\\"Aprendizagem Motora\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend Sativa e a performance em tarefas de atenção sustentada em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Atenção Sustentada\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Energia\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Energia\\\" -> \\\"Atenção Sustentada\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    }
]

# Criar um dataset a partir dos dados
dataset = Dataset.from_dict({"text": [json.dumps(item) for item in train_data]})

# Carregar o modelo e o tokenizer
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Adicionar token de padding ao tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Definir o tamanho máximo da sequência
max_seq_length = 512

# Criar o treinador SFT
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

# Treinar o modelo
trainer.train()

# Salvar o modelo treinado
trainer.save_model("outputs/trained_model")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 124,439,808


Step,Training Loss
1,3.135800
2,3.479400
3,3.404200
4,3.338800
5,2.267500
6,2.181900
7,1.759200
8,1.487800
9,1.207500
10,0.907500


In [13]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
7.059 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 124,439,808


Step,Training Loss
1,0.097200
2,0.093700
3,0.100200
4,0.123000
5,0.088700
6,0.108200
7,0.090000
8,0.091100
9,0.102000
10,0.102300


In [15]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

24.0301 seconds used for training.
0.4 minutes used for training.
Peak reserved memory = 7.059 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 47.864 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, LoraConfig, get_peft_model
import torch

# Carregar o modelo e o tokenizer
model_name = "gpt2"  # Você pode escolher outro modelo, como "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")  # Mover o modelo para a GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Definir o prompt Alpaca
alpaca_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}"""

# Habilitar inferência mais rápida (opcional)
# FastLanguageModel.for_inference(model)

# Preparar a entrada
inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction="Continue the fibonnaci sequence.",
            input="1, 1, 2, 3, 5, 8",
            output="",  # Deixar em branco para geração
        )
    ],
    return_tensors="pt",
).to("cuda")

# Gerar a sequência
with torch.no_grad():  # Desabilitar o cálculo de gradientes durante a geração
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decodificar a saída
print(tokenizer.batch_decode(outputs))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n\n1, 1, 2, 3, 5, 8\n\n### Output:\n\n1, 1, 2, 3, 5, 8\n\n### Error:\n\n1, 1, 2, 3, 5, 8\n\n### Error:\n\n1, 1, 2, 3, 5']


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [21]:
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, TextStreamer
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset
from peft import PeftModel, LoraConfig, get_peft_model
import json

# Base de treinamento inserida diretamente no código
train_data = [
    {
        "instruction": "Analisar",
        "input": "o efeito de um blend de Cannabis Sativa e predominantemente Indica na aprendizagem de idiomas",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Aprendizagem de Idiomas\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Foco\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Ansiedade\\\" [label=\\\"Reduz (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Foco\\\" -> \\\"Aprendizagem de Idiomas\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Ansiedade\\\" -> \\\"Aprendizagem de Idiomas\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o consumo de um blend predominantemente Sativa e a capacidade de memorização de longo prazo",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Memorização de Longo Prazo\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Concentração\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Motivação\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Concentração\\\" -> \\\"Memorização de Longo Prazo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Motivação\\\" -> \\\"Memorização de Longo Prazo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência do teor de THC e CBD em um blend Indica na retenção de informações em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Teor de THC (Blend Indica)\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Teor de CBD (Blend Indica)\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Indica e a ansiedade em estudantes universitários",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Ansiedade\\\" [label=\\\"Reduz (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Ansiedade\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o perfil de terpenos em um blend Sativa e o desempenho em provas de múltipla escolha",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Perfil de Terpenos (Blend Sativa)\\\" -> \\\"Desempenho em Provas\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "o impacto do método de consumo (inalação vs. ingestão) na concentração de canabinoides em um blend Indica e seu efeito na aprendizagem",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Método de Consumo\\\" -> \\\"Concentração de Canabinoides (Blend Indica)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Concentração de Canabinoides (Blend Indica)\\\" -> \\\"Aprendizagem\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o consumo de um blend predominantemente Sativa e a capacidade de foco em estudantes com TDAH",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Foco\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Impulsividade\\\" [label=\\\"Reduz (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Impulsividade\\\" -> \\\"Foco\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o uso de um blend Indica e a retenção de informações em palestras",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Indica\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência da dosagem em um blend predominantemente Indica e seu efeito na capacidade de aprendizagem",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Dosagem (Blend Predominantemente Indica)\\\" -> \\\"Aprendizagem\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend Sativa e a performance em trabalhos em grupo",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Performance em Grupo\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Comunicação\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Comunicação\\\" -> \\\"Performance em Grupo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "o efeito de um blend de Cannabis Sativa e Indica na capacidade de resolução de problemas matemáticos",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Resolução de Problemas\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Pensamento Lógico\\\" [label=\\\"Estimula (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Pensamento Lógico\\\" -> \\\"Resolução de Problemas\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o consumo de um blend predominantemente Sativa e a fluência verbal em debates",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Fluência Verbal\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Articulação de Ideias\\\" [label=\\\"Melhora (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Articulação de Ideias\\\" -> \\\"Fluência Verbal\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência do teor de CBD em um blend Indica na ansiedade durante apresentações em público",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Teor de CBD (Blend Indica)\\\" -> \\\"Ansiedade\\\" [label=\\\"Reduz (PLS-SEM)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Indica e a qualidade do sono em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Qualidade do Sono\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Qualidade do Sono\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o perfil de terpenos em um blend Sativa e a capacidade de concentração em leituras",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Perfil de Terpenos (Blend Sativa)\\\" -> \\\"Concentração\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "o impacto do método de extração na concentração de canabinoides em um blend Indica e seu efeito na memória",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Método de Extração\\\" -> \\\"Concentração de Canabinoides (Blend Indica)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Concentração de Canabinoides (Blend Indica)\\\" -> \\\"Memória\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Sativa e a criatividade em redações",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Criatividade em Redações\\\" [label=\\\"Aumenta (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Pensamento Divergente\\\" [label=\\\"Estimula (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Pensamento Divergente\\\" -> \\\"Criatividade em Redações\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o uso de um blend Indica e a redução do estresse em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Indica\\\" -> \\\"Estresse\\\" [label=\\\"Reduz (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Promove (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Estresse\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência da genética da planta na composição química de um blend predominantemente Sativa e seu efeito na capacidade de foco",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Genética da Planta\\\" -> \\\"Composição Química (Blend Predominantemente Sativa)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Composição Química (Blend Predominantemente Sativa)\\\" -> \\\"Foco\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend Sativa e a performance em atividades que exigem coordenação motora fina, como desenho",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Coordenação Motora Fina\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Concentração\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Concentração\\\" -> \\\"Coordenação Motora Fina\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "o efeito de um blend de Cannabis Sativa e Indica na memória de curto prazo em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Memória de Curto Prazo\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa-Indica\\\" -> \\\"Foco\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Foco\\\" -> \\\"Memória de Curto Prazo\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o consumo de um blend predominantemente Sativa e o desempenho em tarefas de atenção seletiva em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Atenção Seletiva\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Impulsividade\\\" [label=\\\"Reduz (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Impulsividade\\\" -> \\\"Atenção Seletiva\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência do teor de THC e CBD em um blend Indica na capacidade de aprendizagem verbal em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Teor de THC (Blend Indica)\\\" -> \\\"Aprendizagem Verbal\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Teor de CBD (Blend Indica)\\\" -> \\\"Aprendizagem Verbal\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend predominantemente Indica e a performance em tarefas de memória espacial em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Memória Espacial\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Memória Espacial\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o perfil de terpenos em um blend Sativa e a capacidade de resolução de problemas complexos em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Perfil de Terpenos (Blend Sativa)\\\" -> \\\"Resolução de Problemas Complexos\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "o impacto do método de cultivo na concentração de canabinoides em um blend Indica e seu efeito na memória de longo prazo em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Método de Cultivo\\\" -> \\\"Concentração de Canabinoides (Blend Indica)\\\" [label=\\\"Influencia (PLS-SEM)\\\", color=\\\"green\\\"];\\n \\\"Concentração de Canabinoides (Blend Indica)\\\" -> \\\"Memória de Longo Prazo\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o consumo de um blend predominantemente Sativa e a capacidade de aprendizado visuoespacial em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Aprendizado Visuoespacial\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Predominantemente Sativa\\\" -> \\\"Concentração\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Concentração\\\" -> \\\"Aprendizado Visuoespacial\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Analisar",
        "input": "a correlação entre o uso de um blend Indica e a capacidade de retenção de informações em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Indica\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Indica\\\" -> \\\"Relaxamento\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Relaxamento\\\" -> \\\"Retenção de Informações\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    },
    {
        "instruction": "Avaliar",
        "input": "a influência da proporção THC:CBD em um blend predominantemente Indica e seu efeito na capacidade de aprendizagem motora em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Proporção THC:CBD (Blend Predominantemente Indica)\\\" -> \\\"Aprendizagem Motora\\\" [label=\\\"Modula (PLS-SEM)\\\", color=\\\"green\\\"];\\n }"
    },
    {
        "instruction": "Modelar",
        "input": "a relação entre o uso de um blend Sativa e a performance em tarefas de atenção sustentada em estudantes",
        "output": "digraph {\\n node [shape=circle, fontname=\\\"Calibri Bold\\\"];\\n edge [fontname=\\\"Calibri Bold\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Atenção Sustentada\\\" [label=\\\"Melhora (PLS-SEM)\\\", color=\\\"blue\\\"];\\n \\\"Blend Sativa\\\" -> \\\"Energia\\\" [label=\\\"Aumenta (NCA)\\\", color=\\\"blue\\\"];\\n \\\"Energia\\\" -> \\\"Atenção Sustentada\\\" [label=\\\"Influencia (NCA)\\\", color=\\\"blue\\\"];\\n }"
    }
]

# Criar um dataset a partir dos dados
dataset = Dataset.from_dict({"text": [json.dumps(item) for item in train_data]})

# Carregar o modelo e o tokenizer
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Adicionar token de padding ao tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Definir o tamanho máximo da sequência
max_seq_length = 512

# Criar o treinador SFT
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

# Treinar o modelo
trainer.train()

# Salvar o modelo treinado
trainer.save_model("outputs/trained_model")

Map (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 124,439,808


Step,Training Loss
1,3.135800
2,3.479400
3,3.404200
4,3.338300
5,2.267600
6,2.181400
7,1.759600
8,1.487000
9,1.207300
10,0.908000


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris, France. It is located on the Champ de Mars in the 7th arrondissement of Paris, on the left bank of the River Seine. The tower was designed by engineer Gustave Eiffel and his company, and was constructed from 1887 to 18']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [26]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import PeftModel, LoraConfig, get_peft_model
import torch

# Montar o Google Drive (se ainda não estiver montado)
try:
    drive.mount('/content/drive')
except ValueError:
    pass  # Ignorar o erro se o Drive já estiver montado

# ... (código para carregar o modelo, tokenizer, dataset e treinar o modelo)

# Salvar o modelo em formato GGUF no Google Drive
model_save_path = "/content/drive/MyDrive/model.gguf"  # Caminho para salvar o modelo no Google Drive

# Salvar o modelo em formato GGUF
if True:
    # Converter o modelo para o formato adequado para salvamento GGUF
    from transformers import GPT2LMHeadModel

    # Obter o nome ou caminho do modelo base a partir do modelo treinado
    base_model_name_or_path = model.config._name_or_path

    # Carregar o modelo base
    base_model = GPT2LMHeadModel.from_pretrained(base_model_name_or_path)

    # Copiar os pesos do modelo treinado para o modelo base
    base_model.load_state_dict(model.state_dict())

    # Salvar o modelo base em formato GGUF
    base_model.save_pretrained(model_save_path, from_pt=True)

# Opcional: Enviar o modelo para o Hugging Face Hub
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token="")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>